The tensorflow module needs to be imported and used.
This is done from the below given data repository,
one hot refers to representaion of 0-9 digits as one-hot state encoding method of electrical circuits
(Ex : 1 as 000000001, 2 as 000000010 and so on...)

In [9]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot = True)



Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [0]:
n_classes = 10 #Since one class for each of the digits 0-9
batch_size = 128 #feeding 128 values at once through the model

x = tf.placeholder('float', [None, 784]) #the input holder variable, stores in 28*28 pixels as a 1D array (height=None) and size 28*28=784(width=784)
y = tf.placeholder('float')              #the holder variable that'll contain the labels to compare obtained outputs with

In [0]:
#creating convulations
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

#creating a pool
def maxpool2d(x):
    #                        size of window         movement of window
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [0]:
def convolutional_neural_network(x):
    weights = {'W_conv1':tf.Variable(tf.random_normal([5,5,1,32])),   #5*5 convulation , 1 image input and 32 outputs
               'W_conv2':tf.Variable(tf.random_normal([5,5,32,64])),  #5*5 convulation , 32 inputs and 64 outputs
               'W_fc':tf.Variable(tf.random_normal([7*7*64,1024])),   #fully_connected layer , 7*7*64 inputs , 1024 outputs 
               'out':tf.Variable(tf.random_normal([1024, n_classes]))}#1024 inputs , 10 outputs

    #biases exist only for outputs and hence the following values for the layers
    biases = {'b_conv1':tf.Variable(tf.random_normal([32])),
               'b_conv2':tf.Variable(tf.random_normal([64])),
               'b_fc':tf.Variable(tf.random_normal([1024])),
               'out':tf.Variable(tf.random_normal([n_classes]))}

    #reashaping input to a 4D tensor
    x = tf.reshape(x, shape=[-1, 28, 28, 1])
    
    #convulation layer
    conv1 = tf.nn.relu(conv2d(x, weights['W_conv1']) + biases['b_conv1'])  #recitified linear functiopn applied to implement model of (input * weights) + biases
    conv1 = maxpool2d(conv1)  #maxpool formed out of the convulation
   
    #convulation layer 2
    conv2 = tf.nn.relu(conv2d(conv1, weights['W_conv2']) + biases['b_conv2'])  #recitified linear functiopn applied to implement model of (input * weights) + biases
    conv2 = maxpool2d(conv2)   #maxpool formed out of the convulation

    #fully connected layer
    #reshape output of convulation Layer2 to fit fully connected layer
    fc = tf.reshape(conv2,[-1, 7*7*64])
    fc = tf.nn.relu(tf.matmul(fc, weights['W_fc'])+biases['b_fc'])

    output = tf.matmul(fc, weights['out'])+biases['out']

    return output

In [13]:
def train_neural_network(x):
    #prediction stores the one hot encoded results from network model
    prediction = convolutional_neural_network(x)
    #these predictiosn are further comapred to already stored labels
    #declaring the cost functi
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y) )
    #Adan optimizer is a frequently used optimization algo used in deep learning based on stochastic gradient descent, Here it set ot train to minimize cost
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    #epoch is the number of cycles , where one cycle is oen feed forward + one back propagation 
    hm_epochs = 10
    
    #running the TensorFlow seesion
    with tf.Session() as sess:
        #just to initialise all variables
        sess.run(tf.initialize_all_variables())

        #running a loop for 'epoch' number of times
        for epoch in range(hm_epochs):
          
            #to calculate epocch_loss in evry iteration
            epoch_loss = 0
            #dividing number of examples with batch size to figure no. of iterations to go through
            for _ in range(int(mnist.train.num_examples/batch_size)):
              
                #initialising set values of x and y for every chunk of data (every batch)
                epoch_x, epoch_y = mnist.train.next_batch(batch_size)
                _, c = sess.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y})
                epoch_loss += c
            #print epoch_loss to keep track of completion
            print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)
        
        #to check if the one hot encoding of prediction is same as actual label
        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))

        #to compute accuracy as mean of all 'correct' values obtained so far
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:',accuracy.eval({x:mnist.test.images, y:mnist.test.labels}))

#function call to train the netwrok model
train_neural_network(x)

Epoch 0 completed out of 10 loss: 1406681.4548950195
Epoch 1 completed out of 10 loss: 218930.3212738037
Epoch 2 completed out of 10 loss: 129797.74012756348
Epoch 3 completed out of 10 loss: 89777.68626403809
Epoch 4 completed out of 10 loss: 58682.71881000602
Epoch 5 completed out of 10 loss: 42439.06740760803
Epoch 6 completed out of 10 loss: 36668.34846878052
Epoch 7 completed out of 10 loss: 27645.200339399693
Epoch 8 completed out of 10 loss: 24319.295329806395
Epoch 9 completed out of 10 loss: 16152.30190706253
Accuracy: 0.9784
